<a href="https://colab.research.google.com/github/anvargasch/IEStoPBRT/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone the repository

In [ ]:
!git clone https://github.com/anvargasch/IEStoPBRT.git
%cd IEStoPBRT

# Install dependencies

In [ ]:
!pip install numpy matplotlib pyexr opencv-python

# Import the converter

In [ ]:
import sys
sys.path.append("/content/IEStoPBRT")  # adjust if needed

import ies2exr

# Example: convert an IES file into a normalized EXR for PBRT

In [ ]:
filename = "IESfiles/IESsiemens.ies"   # Replace with your own IES file
output   = "IESsiemens_square.exr"

# Generate a square, lat-long, normalized EXR
meta = ies2exr.ies_to_exr(filename, output, size=1024)
print("Metadata:")
for k, v in meta.items():
    print(f"  {k}: {v}")

print("\nRecommended PBRT power (lumens):",
      f"{meta['recommended_power_lm']:.2f}")

# Visualize the generated normalized EXR (R channel)

In [ ]:
import pyexr
import numpy as np
import matplotlib.pyplot as plt

# Read EXR with pyexr (H, W, 3)
img = pyexr.read(output)
r = img[:, :, 0]

plt.figure(figsize=(6, 6))
plt.imshow(r, cmap='gray')
plt.title('Normalized luminous intensity (R channel)')
plt.colorbar(label='Relative intensity (normalized)')
plt.axis('off')
plt.show()

# The EXR file is now ready to be used in PBRT as a goniometric light source.

Use the generated EXR as a lat-long texture and set the PBRT light power to the value given by `meta['recommended_power_lm']` so that the luminous flux in PBRT matches the integrated flux from the original IES file.